# 1.4-Supervised Learning Algorithms Part 1

## 01.Import Libraries

In [ ]:
# Import Libraries
import pandas as pd
import numpy as np
import seaborn as sns
import os
import operator
import matplotlib.pyplot as plt
from sklearn import datasets
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score, confusion_matrix, ConfusionMatrixDisplay
from sklearn.neighbors import KNeighborsClassifier
from sklearn.multioutput import MultiOutputClassifier

## 02.Import Data

In [ ]:
# Create a path for importing
path = r'C:\Users\shrav\_Data_Analysis_CF\ML_ClimateWins_Project'

In [ ]:
#Read in the European weather data.
climate = pd.read_csv(os.path.join(path,'02 Data', 'Original_Data', 'Dataset-weather-prediction-dataset-processed.csv'))
climate

In [ ]:
### Import the pleasant weather data
pleasantweather = pd.read_csv(os.path.join(path, '02 Data', 'Original_Data','Dataset-Answers-Weather_Prediction_Pleasant_Weather.csv'))

In [ ]:
pleasantweather.head()

In [ ]:
climate.columns

In [ ]:
#Reduce to just the temperature variables (excluding Gdansk, Roma,Tours)
df = climate[['DATE', 'MONTH','BASEL_temp_mean','BASEL_temp_min','BASEL_temp_max',        
 'BELGRADE_temp_mean','BELGRADE_temp_min','BELGRADE_temp_max',
 'BUDAPEST_temp_mean', 'BUDAPEST_temp_min', 'BUDAPEST_temp_max',
 'DEBILT_temp_mean','DEBILT_temp_min','DEBILT_temp_max',
 'DUSSELDORF_temp_mean','DUSSELDORF_temp_min','DUSSELDORF_temp_max',
 'HEATHROW_temp_mean','HEATHROW_temp_min','HEATHROW_temp_max',
 'KASSEL_temp_mean','KASSEL_temp_min','KASSEL_temp_max',
 'LJUBLJANA_temp_mean','LJUBLJANA_temp_min','LJUBLJANA_temp_max',
 'MAASTRICHT_temp_mean','MAASTRICHT_temp_min','MAASTRICHT_temp_max',
 'MADRID_temp_mean','MADRID_temp_min','MADRID_temp_max',
 'MUNCHENB_temp_mean','MUNCHENB_temp_min','MUNCHENB_temp_max',
 'OSLO_temp_mean','OSLO_temp_min','OSLO_temp_max',
 'SONNBLICK_temp_mean','SONNBLICK_temp_min','SONNBLICK_temp_max',   
 'STOCKHOLM_temp_mean','STOCKHOLM_temp_min','STOCKHOLM_temp_max',
 'VALENTIA_temp_mean','VALENTIA_temp_min','VALENTIA_temp_max']].copy()

In [ ]:
#Create an 'X' matrix by dropping the unneeded columns.
X = df.drop(['DATE', 'MONTH'], axis=1)
y = pleasantweather.drop(['DATE'], axis=1)

In [ ]:
print(X.shape)
print(y.shape)

In [ ]:
#Split data into training and testing sets.
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [ ]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

In [ ]:
X_train = X_train.to_numpy()
X_test = X_test.to_numpy()
y_train = y_train.to_numpy()
y_test = y_test.to_numpy()

In [ ]:
#Run the model with neighbors equal to 1 to 4, test the accuracy
k_range = np.arange(1,4)
train_acc = np.empty(len(k_range))
test_acc = np.empty(len(k_range))
scores = {}
scores_list = []
for i, k in enumerate(k_range):
    print("i -", i)
    print("k -", k_range)
    knn = KNeighborsClassifier(n_neighbors=k)
    classifier = MultiOutputClassifier(knn, n_jobs=-1) #ADD THIS LINE
    ## Fit the model on the training data.
    knn.fit(X_train, y_train)
    y_pred = knn.predict(X_test)
    train_acc[i] = knn.score(X_train, np.array(y_train)) #CHANGE THIS LINE
    test_acc[i] = knn.score(X_test, np.array(y_test)) #CHANGE THIS LINE

In [ ]:
#plot the relationship between number of neighbors and accuracy
plt.plot(k_range, test_acc, label = 'Test accuracy')
plt.plot(k_range, train_acc, label = 'Train accuracy')
plt.legend()
plt.xlabel('Number of Neighbors')
plt.ylabel('Accuracy')
plt.show

In [ ]:
locations = list({x.split("_")[0] for x in df.columns if x not in ["MONTH", "DATE"]})
locations.sort()
locations

In [ ]:
# Confusion matrix based on the last "k" in the sequence (Test data)
# Assuming y_test is a numpy array, we replace .loc with standard indexing
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

# Assuming labels 'pleasant' and 'unpleasant' are coded as 0 and 1
import pandas as pd

# Assuming locations is already defined
key = "_pleasant_weather"
y_test = pd.DataFrame(y_test, columns=[loc + key for loc in locations])
y_pred = pd.DataFrame(y_pred, columns=locations)

# Updated loop
figure, ax = plt.subplots(3, 5, figsize=(25, 15))
labels = ['pleasant', 'unpleasant']
count = 0

for i in range(3):
    for j in range(5):
        name = locations[count]
        cm = confusion_matrix(y_test[name + key], y_pred[name])
        sns.heatmap(cm, annot=True, fmt='g', ax=ax[i, j])
        
        ax[i, j].set_xlabel('Predicted labels')
        ax[i, j].set_ylabel('True labels')
        ax[i, j].set_title(f"{name} Matrix")
        
        count += 1

plt.tight_layout()
plt.show()

# Specify the path and file name
output_file = r"C:\Users\shrav\_Data_Analysis_CF\ML_ClimateWins_Project\04 Analysis\Visualizations\KNN_model_training_matrix.png"

# Save the figure
figure.savefig(output_file, dpi=300, bbox_inches='tight')

print(f"Figure saved successfully at: {output_file}")


In [ ]:
pleasantweather['SONNBLICK_pleasant_weather'].value_counts(dropna=False)

In [ ]:
pleasantweather['VALENTIA_pleasant_weather'].value_counts(dropna=False)

### note: Error occuring with SONNBLICK_pleasant_weather_answer due to having no ('pleasant weather attributes' = 1)